In [10]:
import os, random
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm, trange

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from NSR.model import *
from NSR.utils import *

import warnings
warnings.filterwarnings(action='ignore')

# Pytorch determistic
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [11]:
def split_behavior(df, behavior):
    behavior_index_value = df[df['0_behavior'] == behavior].index.values
    idx_for_behavior = np.where(behavior_index_value[1:] - behavior_index_value[:-1] > 1)[0] + 1
    return idx_for_behavior

def get_scenario(data):   
    prepare_idx  = split_behavior(data, 'prepare')
    takeoff_idx  = split_behavior(data, 'takeoff')
    forward_idx  = split_behavior(data, 'forward')
    backward_idx = split_behavior(data, 'backward')
    right_idx    = split_behavior(data, 'right')
    left_idx    = split_behavior(data, 'left')
    up_idx       = split_behavior(data, 'up')
    down_idx     = split_behavior(data, 'down')
    land_idx     = split_behavior(data, 'land')
    behavior_idx = [prepare_idx, takeoff_idx, forward_idx, backward_idx, right_idx, left_idx, up_idx, down_idx, land_idx] 
    scenarios = [np.sum(np.asarray(behavior_idx)[:, each_idx]) for each_idx in range(len(prepare_idx))]
    scenarios.append(len(data))
    
    scenario_list = [None for _ in range(len(data))]
    start = 0
    for scenario_num, scenario_idx in enumerate(scenarios):
        for i in range(start, scenario_idx):
            scenario_list[i] = scenario_num
        start = scenario_idx        
    return scenario_list, scenarios

def get_subsystems(all_columns, target_colum):
    system_cate = [each.split('.')[0] for each in all_columns]
    selected_subsystem = all_columns[np.array(system_cate) == target_colum]
    return list(selected_subsystem) + ['timestamp']

def normalize_v1(data, ison_list, train=50, val=5, test=10):
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))
    
    print(random_scenario)
    
    train_scenario = random_scenario[:train]
    val_scenario   = random_scenario[train:train+val]
    test_scenario  = random_scenario[train+val:train+val+test]
    
    train_scaler   = []
    val_scaler   = []
    test_scaler   = []
    
    train_normalized = []
    val_normalized   = []
    test_normalized  = []
    
    for i in np.unique(data['scenario']):
        scaler          = CustomScaler()
        target_data     = data[data['scenario'] == i].drop(['scenario'], axis=1)
        not_norm_column = ison_list.copy()
        not_norm_column.append('0_behavior')
        target_column   = set(target_data.keys()).difference(set(not_norm_column))
        
        if i in train_scenario:
            train_normalized.append(scaler.fit_transform(target_data, target_column))
            train_scaler.append(scaler)
        elif i in val_scenario:
            val_normalized.append(scaler.fit_transform(target_data, target_column))
            val_scaler.append(scaler)
        elif i in test_scenario:
            test_normalized.append(scaler.fit_transform(target_data, target_column))
            test_scaler.append(scaler)
    
    train_norm = np.concatenate(train_normalized)
    val_norm = np.concatenate(val_normalized)
    test_norm = np.concatenate(test_normalized)

    return [train_norm, train_scenario, train_scaler],[val_norm, val_scenario, val_scaler],[test_norm, test_scenario, test_scaler]

def normalize_v2(data, ison_list, train=50, val=5, test=10):
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))
    
    train_scenario = random_scenario[:train]
    val_scenario   = random_scenario[train:train+val]
    test_scenario  = random_scenario[train+val:train+val+test]
    
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))

    train_scenario = random_scenario[:50]
    val_scenario   = random_scenario[50:55]
    test_scenario  = random_scenario[55:65]

    train_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in train_scenario]
    train_data = data.iloc[sorted(np.hstack(train_idx))]

    val_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in val_scenario]
    val_data = data.iloc[sorted(np.hstack(val_idx))]

    test_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in test_scenario]
    test_data = data.iloc[sorted(np.hstack(test_idx))]

    not_norm_column = ison_list.copy()
    not_norm_column.append('0_behavior')
    not_norm_column.append('scenario')
    target_column   = set(train_data.keys()).difference(set(not_norm_column))

    scaler = CustomScaler()
    scaler.fit_transform(train_data, target_column)
    scaler.transform(val_data, target_column)
    scaler.transform(test_data, target_column)
    
    return [train_data, val_data, test_data],\
    [train_scenario, val_scenario, test_scenario],\
    scaler

def get_data():
    data = pd.read_csv('ss/preprocessing/px4_normal_nanprocessed.csv')
    list_of_scenario, scenario_idx = get_scenario(data)
    data['scenario'] = list_of_scenario

    # 기타변수 제거
    ignore_colums = ['0_environment', '0_target', 'name', 'time', 'timestamp']

    # time 변수 위치 파악 및 제거
    columns = list(data.keys())
    time_list = [each for each in list(data.keys()) if each.find('time')>0]
    data.drop(ignore_colums+time_list, axis=1, inplace=True)

    # ison 변수 위치 파악
    ison_list = [each for each in list(data.keys()) if each.find('ison')>0]
    ison_dict = {key: np.where(np.array(list(data.keys())) == key)[0][0] for key in ison_list}

    # Behavior 카테고리 변환
    behaviors = ['prepare', 'takeoff', 'forward', 'backward', 'up', 'down', 'right', 'left', 'land']
    label_dict = Vocabulary()
    for each_behavior in behaviors:
        label_dict.add_word(each_behavior)
    data['0_behavior'] = data['0_behavior'].map(label_dict.word2idx)

    (train_data, val_data, test_data), (train_scenario, val_scenario, test_scenario), scaler = normalize_v2(data, ison_list)

    # 시나리오별 성능 확인용
    train_scenario_index = train_data['scenario']
    val_scenario_index   = val_data['scenario']
    test_scenario_index  = test_data['scenario']

    X_train, y_train = split_series_v2(train_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_val, y_val     = split_series_v2(val_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_test, y_test   = split_series_v2(test_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)

    X_train_drop = X_train[:, :, 1:]
    X_val_drop   = X_val[:, :, 1:]
    X_test_drop  = X_test[:, :, 1:]
    
    res_list = list(train_data.drop(['0_behavior', 'scenario'], axis=1).columns)
    res_list_ison_out = [each for each in res_list if not each.find('ison')>0]
    res_list_ison_out_dict = {each:idx for idx, each in enumerate(res_list_ison_out)}

    meta = [each.split('.') for each in res_list_ison_out]
    meta_head = np.array([each.split('.')[0] for each in res_list_ison_out])
    super_column = np.unique(meta_head)

    column_idx = {}
    for each_column in super_column:
        column_idx[each_column] = list(np.where(meta_head == each_column)[0])
    
    return X_train_drop, y_train, X_val_drop, y_val, X_test_drop, y_test, column_idx

In [16]:
def train(model, schedule, EPOCH, train_dataloader, val_dataloader, SAVE_PATH, model_type, device, len_val):
    print('train path: ', SAVE_PATH+'/res.csv')
    
    scaler = torch.cuda.amp.GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterionMSE = nn.MSELoss()
    criterionCE = nn.CrossEntropyLoss()
    if schedule:
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40], gamma=0.1)
        
    res = {
        'train_mse':[],
        'train_ce':[],
        'train_total':[],

        'val_mse':[],
        'val_ce':[],
        'val_total':[],
        'val_acc':[],

        'best_total':[],
    }
    
    best_total = np.inf
    best_epoch = 0
    
    for epoch in trange(EPOCH):
        # Training Procedure
        train_running_step = len(train_dataloader)
        #print(f'runnnig step: {train_running_step}')

        model.train();
        train_mse   = 0
        train_ce    = 0
        train_total = 0
        for train_idx, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            if model_type == 'DNN':
                labels_state = labels[:, 1:].to(device)
                labels_behavior = labels[:, 0].type(torch.LongTensor).to(device)
            else:
                labels_state = labels[:, :, 1:].to(device)
                labels_behavior = labels[:, :, 0].type(torch.LongTensor).to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                if model_type in ['RNN', 'LSTM', 'GRU']:
                    outputs = outputs[0][:, -1, :].unsqueeze(1)
                #print(outputs[0][:, -1, :].unsqueeze(1).shape, labels_state.shape, labels_behavior.shape)
                
                if model_type == 'DNN':
                    loss_mse = criterionMSE(outputs[:, :-9], labels_state)     # 기체 상태 MSE 예측
                    loss_ce = criterionCE(outputs[:, -9:], labels_behavior)    # behavior 확률 변수
                else:
                    if model_type[:7] == 'Stacked': 
                        outputs = outputs.unsqueeze(1)
                    loss_mse = criterionMSE(outputs[:, :, :-9], labels_state)
                    loss_ce = criterionCE(outputs[:, :, -9:].squeeze(1), labels_behavior.squeeze(1))   

                loss = loss_mse + loss_ce
            loss.backward()
            optimizer.step()

            train_mse += loss_mse.item()
            train_ce += loss_ce.item()
            train_total += loss.item()

            # Output training stats
    #         if (train_idx % int(train_running_step/2)) == 0:
    #             print(f'[{epoch}/{EPOCH}], \n\t[Single|Avg] = [{round(loss.item(), 5)}|{round(train_total/(train_idx+1), 5)}]')
        train_mse   = train_mse/train_running_step
        train_ce    = train_ce/train_running_step
        train_total = train_total/train_running_step

        # validation Procedure
        val_running_step = len(val_dataloader)
        #print(f'runnnig step: {val_running_step}')

        # Accuracy구현해야됨
        model.eval();
        val_mse   = 0
        val_ce    = 0 
        val_total = 0
        val_acc   = 0
        with torch.no_grad():
            for val_idx, (inputs, labels) in enumerate(val_dataloader):
                inputs = inputs.to(device)
                if model_type == 'DNN':
                    labels_state = labels[:, 1:].to(device)
                    labels_behavior = labels[:, 0].type(torch.LongTensor).to(device)
                else:
                    labels_state = labels[:, :, 1:].to(device)
                    labels_behavior = labels[:, :, 0].type(torch.LongTensor).to(device)

                outputs = model(inputs)
                if model_type in ['RNN', 'LSTM', 'GRU']:
                    outputs = outputs[0][:, -1, :].unsqueeze(1)
                if model_type == 'DNN':
                    loss_mse = criterionMSE(outputs[:, :-9], labels_state)     # 기체 상태 MSE 예측
                    loss_ce = criterionCE(outputs[:, -9:], labels_behavior)    # behavior 확률 변수
                    val_acc += (outputs[:, -9:].argmax(axis=1) == labels_behavior).sum().item()
                else:
                    if model_type[:7] == 'Stacked':  
                        outputs = outputs.unsqueeze(1)
                    loss_mse = criterionMSE(outputs[:, :, :-9], labels_state)
                    loss_ce = criterionCE(outputs[:, :, -9:].squeeze(1), labels_behavior.squeeze(1))  
                    val_acc += (outputs[:,:, -9:].argmax(axis=2) == labels_behavior).sum().item()

                val_mse += loss_mse.item()
                val_ce  += loss_ce.item()
                val_total += val_ce + val_mse
 
        val_mse   = val_mse/val_running_step
        val_ce    = val_ce/val_running_step
        val_total = val_total/val_running_step
        val_acc   = val_acc/len_val

        if val_total < best_total:
            best_epoch = epoch
            #print(f'\t Best MSE changed [{round(best_total, 5)} ---> {round(val_total, 5)}]')
            save('best', epoch, SAVE_PATH, model, optimizer)     
            best_total = val_total
        save('last', epoch, SAVE_PATH, model, optimizer)
        #print(f'\t Validaition [Current|Best]: [{round(val_mse, 5)}| {round(best_total, 5)}]')

        res['train_mse'].append(train_mse)
        res['train_ce'].append(train_ce)
        res['train_total'].append(train_total)

        res['val_mse'].append(val_mse)
        res['val_ce'].append(val_ce)
        res['val_total'].append(val_total)
        res['val_acc'].append(val_acc)

        res['best_total'].append(best_total)
        pd.DataFrame(res).to_csv(SAVE_PATH+'/res.csv', index=False)

        if schedule:
            scheduler.step()
    print('best_Epoch: ', best_epoch)

def test(model, SAVE_PATH, model_type, test_dataloader, device, column_idx, len_test):
    model.load_state_dict(torch.load(os.path.join(SAVE_PATH, 'best.path.tar'))['model_state_dict'])
    
    criterion = nn.MSELoss()
    softmax   = nn.Softmax()
    model.eval();
    #test_results = {}
    mse = []
    logits = []
    acc = 0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            if model_type == 'DNN':
                labels_state = labels[:, 1:].to(device)
                labels_behavior = labels[:, 0].type(torch.LongTensor).to(device)
            else:
                labels_state = labels[:, :, 1:].to(device)
                labels_behavior = labels[:, :, 0].type(torch.LongTensor).to(device)
            
            
            outputs = model(inputs)
            if model_type in ['RNN', 'LSTM', 'GRU']:
                    outputs = outputs[0][:, -1, :].unsqueeze(1)
            outputs=outputs.squeeze(1)
            mse.extend(((outputs[:, :-9] - labels_state.squeeze(1))**2).detach().cpu().numpy())
            logits.extend(softmax(outputs[:, -9:]).detach().cpu().numpy())

            if model_type == 'DNN':
                acc += (outputs[:, -9:].argmax(1) == labels_behavior).sum().item()
            else:
                acc += (outputs[:, -9:].argmax(1) == labels_behavior.squeeze(1)).sum().item()

    mse = np.array(mse).mean(axis=0)

    results = {}
    results['accuracy'] = acc/len_test
    for idx in column_idx:
        results[idx] = [mse[column_idx[idx]].mean()]
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(SAVE_PATH, 'test.csv'), index=False)
    print('test save path: ', os.path.join(SAVE_PATH, 'test.csv'))
    print('\n####################################################################\n')
        
    
                          
def experiment(model_type, hid_dim, n_layers, window_size, pred_len, EPOCH, schedule, X_train, y_train, X_val, y_val, X_test, y_test, column_idx):
    IN_FEAT     = X_train.shape[-1]
    OUT_FEAT    = 170 # y_train.shape[-1] + 8  # number of classes
    IN_SEQ      = window_size
    OUT_SEQ     = pred_len
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    if model_type == 'DNN':
        flatten = True
        model = DNN(input_size = IN_FEAT*IN_SEQ, hidden_size = hid_dim, output_size = OUT_FEAT*OUT_SEQ).to(device)
    else: 
        flatten = False
        if model_type[:7] == 'Stacked':
            model = Stacked(model_type[7:], IN_FEAT, hid_dim, OUT_FEAT, 4).to(device)
        elif model_type[:7] == 'Seq2Seq':
            model = Seq2Seq(input_dim=IN_FEAT, output_dim=OUT_FEAT, hid_dim=hid_dim, n_layers=n_layers, model_type=model_type[8:], pred_len=OUT_SEQ).to(device)
        else:
            model = eval('nn.{}(IN_FEAT, OUT_FEAT, n_layers, batch_first=True).to(device)'.format(model_type))
        
    
    trainSet = CustomDataset(X_train, y_train, flatten=flatten)
    valSet   = CustomDataset(X_val, y_val, flatten=flatten)
    testSet  = CustomDataset(X_test, y_test, flatten=flatten)

    train_dataloader = torch.utils.data.DataLoader(trainSet, batch_size=64, shuffle=True, drop_last=True, num_workers=2, pin_memory=True)
    val_dataloader   = torch.utils.data.DataLoader(valSet, batch_size=64, shuffle=False, drop_last=False, num_workers=2, pin_memory=True)
    test_dataloader   = torch.utils.data.DataLoader(testSet, batch_size=64, shuffle=False, drop_last=False, num_workers=2, pin_memory=True)
    
    SAVE_PATH   = os.path.join('weights', '_'.join([model_type, str(window_size), str(pred_len), str(hid_dim), str(n_layers)]))

    if schedule:
        SAVE_PATH   = os.path.join(['weights', 'scheculer', '_'.join([model_type, window_size, pred_len])])
    os.makedirs(SAVE_PATH, exist_ok=True)
    
    train(model, schedule, EPOCH, train_dataloader, val_dataloader, SAVE_PATH, model_type, device, len(valSet))
    
    test(model, SAVE_PATH, model_type, test_dataloader, device, column_idx, len(testSet))
    
    if device == 'cuda':
        torch.cuda.empty_cache()

In [17]:
models = ['DNN', 'Seq2Seq_RNN', 'Seq2Seq_LSTM', 'Seq2Seq_GRU']
#models = ['RNN']
epoch = 100
hid_dim = 256
n_layers=4
window_size = 10
pred_len = 1
schedule = False


X_train, y_train, X_val, y_val, X_test, y_test, column_idx = get_data()
for model_type in models:
    experiment(model_type, hid_dim, n_layers, window_size, pred_len, epoch, schedule, X_train, y_train, X_val, y_val, X_test, y_test, column_idx)
    

train path:  weights\DNN_10_1_256_4/res.csv


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [11:58<00:00,  7.18s/it]


best_Epoch:  86
test save path:  weights\DNN_10_1_256_4\test.csv

####################################################################

train path:  weights\Seq2Seq_RNN_10_1_256_4/res.csv


  0%|                                                                                          | 0/100 [00:03<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

# 최소 최대 column 뽑기

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('weights_fix_random/DNN_10_1_256_4/test.csv')
# df = pd.read_csv('weights_fix_random/Seq2Seq_RNN_10_1_256_4/test.csv')
df = df.drop(['accuracy'], axis=1)
col = df.keys()
print(col[np.array(df).argmin()], col[np.array(df).argmax()])


# torch info

In [ ]:
dnn : torch.Size([64, 170]) torch.Size([64, 161]) torch.Size([64])
seq2seq : torch.Size([64, 1, 170]) torch.Size([64, 1, 161]) torch.Size([64, 1])

In [8]:
from torchinfo import summary
model = Seq2Seq(input_dim=181, output_dim=170, hid_dim=256, n_layers=4, model_type='LSTM', pred_len=1).cuda()
summary(model, (64, 10, 181))

Layer (type:depth-idx)                   Output Shape              Param #
Seq2Seq                                  --                        --
├─Encoder: 1-1                           [64, 10, 256]             --
│    └─LSTM: 2-1                         [64, 10, 256]             2,028,544
├─DecoderCell: 1-2                       [64, 1, 256]              --
│    └─LSTM: 2-2                         [64, 1, 256]              2,105,344
├─Linear: 1-3                            [64, 1, 170]              43,690
Total params: 4,177,578
Trainable params: 4,177,578
Non-trainable params: 0
Total mult-adds (G): 1.44
Input size (MB): 0.46
Forward/backward pass size (MB): 1.53
Params size (MB): 16.71
Estimated Total Size (MB): 18.70

In [9]:
model = DNN(input_size = 1810, hidden_size = 256, output_size = 170).cuda()
summary(model, (64, 1810))

Layer (type:depth-idx)                   Output Shape              Param #
DNN                                      --                        --
├─Sequential: 1-1                        [64, 170]                 --
│    └─Linear: 2-1                       [64, 256]                 463,616
│    └─ReLU: 2-2                         [64, 256]                 --
│    └─Linear: 2-3                       [64, 256]                 65,792
│    └─ReLU: 2-4                         [64, 256]                 --
│    └─Linear: 2-5                       [64, 256]                 65,792
│    └─ReLU: 2-6                         [64, 256]                 --
│    └─Linear: 2-7                       [64, 170]                 43,690
Total params: 638,890
Trainable params: 638,890
Non-trainable params: 0
Total mult-adds (M): 40.89
Input size (MB): 0.46
Forward/backward pass size (MB): 0.48
Params size (MB): 2.56
Estimated Total Size (MB): 3.50

# 고정 -> 가변

In [7]:
def normalize_v3(data, ison_list, train=50, val=5, test=10):
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))
    
    train_scenario = random_scenario[:train]
    val_scenario   = random_scenario[train:train+val]
    test_scenario  = random_scenario[train+val:train+val+test]

    train_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in train_scenario]
    train_data = data.iloc[sorted(np.hstack(train_idx))]

    val_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in val_scenario]
    val_data = data.iloc[sorted(np.hstack(val_idx))]

    test_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in test_scenario]
    test_data = data.iloc[sorted(np.hstack(test_idx))]

    not_norm_column = ison_list.copy()
    not_norm_column.append('0_behavior')
    not_norm_column.append('scenario')
    target_column   = set(train_data.keys()).difference(set(not_norm_column))

    scaler = CustomScaler()
    scaler.fit_transform(train_data, target_column)
    scaler.transform(val_data, target_column)
    scaler.transform(test_data, target_column)
    
    return [train_data, val_data, test_data],\
    [train_scenario, val_scenario, test_scenario],\
    scaler

def get_data_fixed_random():
    # original dataframe
    fixed_nsr = pd.read_csv('px4-fixed-normal-preprocessed-NSR.csv')

    # original dataframe
    random_dash = pd.read_csv('px4-random-normal-preprocessed-dash.csv')

    # 기타변수 제거 + time 변수 위치 파악 및 제거
    ignore_colum        = ['0_environment', '0_target', 'name', 'time', 'timestamp']
    columns             = list(fixed_nsr.keys())
    time_related_column = [each for each in list(fixed_nsr.keys()) if each.find('time')>0]
    ignore_colum.extend(time_related_column)

    fixed_nsr_dropped = fixed_nsr.drop(ignore_colum, axis=1)
    random_dash_dropped = random_dash.drop(ignore_colum, axis=1)

    # ison 변수 위치 파악
    ison_list = [each for each in list(fixed_nsr_dropped.keys()) if each.find('ison')>0]
    ison_dict = {key: np.where(np.array(list(fixed_nsr_dropped.keys())) == key)[0][0] for key in ison_list}

    # Behavior 카테고리 변환 for fixed-nsr-dropped  & random-dash-dropped
    behaviors  = ['prepare', 'takeoff', 'forward', 'backward', 'up', 'down', 'right', 'left', 'land']
    label_dict = Vocabulary()
    for each_behavior in behaviors:
        label_dict.add_word(each_behavior)

    fixed_nsr_dropped['0_behavior']   = fixed_nsr_dropped['0_behavior'].map(label_dict.word2idx)
    random_dash_dropped['0_behavior'] = random_dash_dropped['0_behavior'].map(label_dict.word2idx)
        
    # Fixed-NSR 나누기
    (train_data1, val_data1, test_data1), (train_scenario1, val_scenario1, test_scenario1), scaler1 = normalize_v3(fixed_nsr_dropped, ison_list)

    # Random-DASH 나누기
    origin = int(len(np.unique(random_dash_dropped['scenario'])))
    test  = int(origin * 0.2)
    train = int((origin-test)*0.9)
    val   = int((origin-test)*0.1)
    (train_data2, val_data2, test_data2), (train_scenario2, val_scenario2, test_scenario2), scaler2 = normalize_v3(random_dash_dropped, ison_list, train=train, val=val, test=test)

    # Fixed-NSR: Time-Series Split 진행
    X_train_nsr, y_train_nsr = split_series_v2(train_data1.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_val_nsr, y_val_nsr     = split_series_v2(val_data1.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_test_nsr, y_test_nsr   = split_series_v2(test_data1.drop(['scenario'], axis=1).values, ison_dict, 10, 1)

    # Random-DASH: Time-Series Split 진행
    X_train_dash, y_train_dash = split_series_v2(train_data2.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_val_dash, y_val_dash     = split_series_v2(val_data2.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
    X_test_dash, y_test_dash   = split_series_v2(test_data2.drop(['scenario'], axis=1).values, ison_dict, 10, 1)

    # Fixed-NSR: input 데이터 맨 앞 behavior 칼럼 제외 (behavior는 맨 뒤로 one-hot encoding 되어있음)
    X_train_drop_nsr = X_train_nsr[:, :, 1:]
    X_val_drop_nsr   = X_val_nsr[:, :, 1:]
    X_test_drop_nsr  = X_test_nsr[:, :, 1:]

    # Random-DASH: input 데이터 맨 앞 behavior 칼럼 제외 (behavior는 맨 뒤로 one-hot encoding 되어있음)
    X_train_drop_dash = X_train_dash[:, :, 1:]
    X_val_drop_dash   = X_val_dash[:, :, 1:]
    X_test_drop_dash  = X_test_dash[:, :, 1:]
    
    res_list = list(train_data1.drop(['0_behavior', 'scenario'], axis=1).columns)
    res_list_ison_out = [each for each in res_list if not each.find('ison')>0]
    res_list_ison_out_dict = {each:idx for idx, each in enumerate(res_list_ison_out)}

    meta = [each.split('.') for each in res_list_ison_out]
    meta_head = np.array([each.split('.')[0] for each in res_list_ison_out])
    super_column = np.unique(meta_head)

    column_idx = {}
    for each_column in super_column:
        column_idx[each_column] = list(np.where(meta_head == each_column)[0])
    
    return X_train_drop_nsr, y_train_nsr, X_train_drop_dash, y_train_dash, X_val_drop_nsr, y_val_nsr, X_val_drop_dash, y_val_dash, X_test_drop_nsr, y_test_nsr, X_test_drop_dash, y_test_dash, column_idx

def test_random(model, SAVE_PATH, model_type, test_dataloader, device, column_idx, len_test):
    model.load_state_dict(torch.load(os.path.join(SAVE_PATH, 'best.path.tar'))['model_state_dict'])
    
    criterion = nn.MSELoss()
    softmax   = nn.Softmax()
    model.eval();
    #test_results = {}
    mse = []
    logits = []
    acc = 0
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            if model_type == 'DNN':
                labels_state = labels[:, 1:].to(device)
                labels_behavior = labels[:, 0].type(torch.LongTensor).to(device)
            else:
                labels_state = labels[:, :, 1:].to(device)
                labels_behavior = labels[:, :, 0].type(torch.LongTensor).to(device)
            
            
            outputs = model(inputs)
            if model_type in ['RNN', 'LSTM', 'GRU']:
                    outputs = outputs[0][:, -1, :].unsqueeze(1)
            outputs=outputs.squeeze(1)
            mse.extend(((outputs[:, :-9] - labels_state.squeeze(1))**2).detach().cpu().numpy())
            logits.extend(softmax(outputs[:, -9:]).detach().cpu().numpy())

            if model_type == 'DNN':
                acc += (outputs[:, -9:].argmax(1) == labels_behavior).sum().item()
            else:
                acc += (outputs[:, -9:].argmax(1) == labels_behavior.squeeze(1)).sum().item()

    mse = np.array(mse).mean(axis=0)

    results = {}
    results['accuracy'] = acc/len_test
    for idx in column_idx:
        results[idx] = [mse[column_idx[idx]].mean()]
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(SAVE_PATH, 'test_random.csv'), index=False)
    print('test save path: ', os.path.join(SAVE_PATH, 'test_random.csv'))
    print('\n####################################################################\n')

def experiment_only_test(model_type, hid_dim, n_layers, window_size, pred_len, EPOCH, schedule, X_train, y_train, X_val, y_val, X_test, y_test, column_idx):
    IN_FEAT     = X_train.shape[-1]
    OUT_FEAT    = 170 # y_train.shape[-1] + 8  # number of classes
    IN_SEQ      = window_size
    OUT_SEQ     = pred_len
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    if model_type == 'DNN':
        flatten = True
        model = DNN(input_size = IN_FEAT*IN_SEQ, hidden_size = hid_dim, output_size = OUT_FEAT*OUT_SEQ).to(device)
    else: 
        flatten = False
        if model_type[:7] == 'Stacked':
            model = Stacked(model_type[7:], IN_FEAT, hid_dim, OUT_FEAT, 4).to(device)
        elif model_type[:7] == 'Seq2Seq':
            model = Seq2Seq(input_dim=IN_FEAT, output_dim=OUT_FEAT, hid_dim=hid_dim, n_layers=n_layers, model_type=model_type[8:], pred_len=OUT_SEQ).to(device)
        else:
            model = eval('nn.{}(IN_FEAT, OUT_FEAT, n_layers, batch_first=True).to(device)'.format(model_type))
        
    
    testSet  = CustomDataset(X_test, y_test, flatten=flatten)
    test_dataloader   = torch.utils.data.DataLoader(testSet, batch_size=64, shuffle=False, drop_last=False, num_workers=2, pin_memory=True)
    
    SAVE_PATH   = os.path.join('weights', '_'.join([model_type, str(window_size), str(pred_len), str(hid_dim), str(n_layers)]))

    if schedule:
        SAVE_PATH   = os.path.join(['weights', 'scheculer', '_'.join([model_type, window_size, pred_len])])
    os.makedirs(SAVE_PATH, exist_ok=True)
    
    test_random(model, SAVE_PATH, model_type, test_dataloader, device, column_idx, len(testSet))
    
    if device == 'cuda':
        torch.cuda.empty_cache()

In [11]:
models = ['DNN', 'Seq2Seq_RNN', 'Seq2Seq_LSTM', 'Seq2Seq_GRU']
#models = ['RNN']
epoch = 100
hid_dim = 256
n_layers=4
window_size = 10
pred_len = 1
schedule = False

# Pytorch determistic
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

X_train_drop_nsr, y_train_nsr, X_train_drop_dash, y_train_dash, X_val_drop_nsr, y_val_nsr, X_val_drop_dash, y_val_dash, X_test_drop_nsr, y_test_nsr, X_test_drop_dash, y_test_dash, column_idx = get_data_fixed_random()

for model_type in models:
    experiment_only_test(model_type, hid_dim, n_layers, window_size, pred_len, epoch, schedule, X_train_drop_dash, y_train_dash, X_val_drop_dash, y_val_dash, X_test_drop_dash, y_test_dash, column_idx)
    

test save path:  weights\DNN_10_1_256_4\test_random.csv

####################################################################

test save path:  weights\Seq2Seq_RNN_10_1_256_4\test_random.csv

####################################################################

test save path:  weights\Seq2Seq_LSTM_10_1_256_4\test_random.csv

####################################################################

test save path:  weights\Seq2Seq_GRU_10_1_256_4\test_random.csv

####################################################################



# 고정&가변 -> 고정&가변

In [12]:
def experiment_fix_random(model_type, hid_dim, n_layers, window_size, pred_len, EPOCH, schedule, X_train, y_train, X_val, y_val, X_test, y_test, column_idx):
    IN_FEAT     = X_train.shape[-1]
    OUT_FEAT    = 170 # y_train.shape[-1] + 8  # number of classes
    IN_SEQ      = window_size
    OUT_SEQ     = pred_len
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    if model_type == 'DNN':
        flatten = True
        model = DNN(input_size = IN_FEAT*IN_SEQ, hidden_size = hid_dim, output_size = OUT_FEAT*OUT_SEQ).to(device)
    else: 
        flatten = False
        if model_type[:7] == 'Stacked':
            model = Stacked(model_type[7:], IN_FEAT, hid_dim, OUT_FEAT, 4).to(device)
        elif model_type[:7] == 'Seq2Seq':
            model = Seq2Seq(input_dim=IN_FEAT, output_dim=OUT_FEAT, hid_dim=hid_dim, n_layers=n_layers, model_type=model_type[8:], pred_len=OUT_SEQ).to(device)
        else:
            model = eval('nn.{}(IN_FEAT, OUT_FEAT, n_layers, batch_first=True).to(device)'.format(model_type))
        
    
    trainSet = CustomDataset(X_train, y_train, flatten=flatten)
    valSet   = CustomDataset(X_val, y_val, flatten=flatten)
    testSet  = CustomDataset(X_test, y_test, flatten=flatten)

    train_dataloader = torch.utils.data.DataLoader(trainSet, batch_size=64, shuffle=True, drop_last=True, num_workers=2, pin_memory=True)
    val_dataloader   = torch.utils.data.DataLoader(valSet, batch_size=64, shuffle=False, drop_last=False, num_workers=2, pin_memory=True)
    test_dataloader   = torch.utils.data.DataLoader(testSet, batch_size=64, shuffle=False, drop_last=False, num_workers=2, pin_memory=True)
    
    SAVE_PATH   = os.path.join('weights_fix_random', '_'.join([model_type, str(window_size), str(pred_len), str(hid_dim), str(n_layers)]))

    if schedule:
        SAVE_PATH   = os.path.join(['weights_fix_random', 'scheculer', '_'.join([model_type, window_size, pred_len])])
    os.makedirs(SAVE_PATH, exist_ok=True)
    
    train(model, schedule, EPOCH, train_dataloader, val_dataloader, SAVE_PATH, model_type, device, len(valSet))
    
    test(model, SAVE_PATH, model_type, test_dataloader, device, column_idx, len(testSet))
    
    if device == 'cuda':
        torch.cuda.empty_cache()

In [13]:
models = ['DNN', 'Seq2Seq_RNN', 'Seq2Seq_LSTM', 'Seq2Seq_GRU']
#models = ['RNN']
epoch = 100
hid_dim = 256
n_layers=4
window_size = 10
pred_len = 1
schedule = False

# Pytorch determistic
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

X_train_drop_nsr, y_train_nsr, X_train_drop_dash, y_train_dash, X_val_drop_nsr, y_val_nsr, X_val_drop_dash, y_val_dash, X_test_drop_nsr, y_test_nsr, X_test_drop_dash, y_test_dash, column_idx = get_data_fixed_random()

X_train = np.vstack([X_train_drop_nsr, X_train_drop_dash])
y_train = np.vstack([y_train_nsr, y_train_dash])

X_val = np.vstack([X_val_drop_nsr, X_val_drop_dash])
y_val = np.vstack([y_val_nsr, y_val_dash])

X_test = np.vstack([X_test_drop_nsr, X_test_drop_dash])
y_test = np.vstack([y_test_nsr, y_test_dash])

for model_type in models:
    experiment_fix_random(model_type, hid_dim, n_layers, window_size, pred_len, epoch, schedule, X_train, y_train, X_val, y_val, X_test, y_test, column_idx)
    

train path:  weights_fix_random\DNN_10_1_256_4/res.csv


100%|████████████████████████████████████████| 100/100 [12:19<00:00,  7.40s/it]


best_Epoch:  4
test save path:  weights_fix_random\DNN_10_1_256_4\test.csv

####################################################################

train path:  weights_fix_random\Seq2Seq_RNN_10_1_256_4/res.csv


100%|████████████████████████████████████████| 100/100 [20:07<00:00, 12.07s/it]


best_Epoch:  44
test save path:  weights_fix_random\Seq2Seq_RNN_10_1_256_4\test.csv

####################################################################

train path:  weights_fix_random\Seq2Seq_LSTM_10_1_256_4/res.csv


100%|████████████████████████████████████████| 100/100 [22:33<00:00, 13.53s/it]


best_Epoch:  30
test save path:  weights_fix_random\Seq2Seq_LSTM_10_1_256_4\test.csv

####################################################################

train path:  weights_fix_random\Seq2Seq_GRU_10_1_256_4/res.csv


100%|████████████████████████████████████████| 100/100 [22:02<00:00, 13.23s/it]


best_Epoch:  13
test save path:  weights_fix_random\Seq2Seq_GRU_10_1_256_4\test.csv

####################################################################

